In [ ]:
!pip install pyservicebinding
!pip install psycopg2-binary

In [ ]:
import os
import pyservicebinding
from pyservicebinding import binding

if 'SERVICE_BINDING_ROOT' in os.environ:
    print('DB binding ok')
else:
    print('DB binding missing')

In [ ]:
sb = binding.ServiceBinding()
bindings_list = sb.bindings('postgresql', 'CockroachDB Cloud')

In [ ]:
import psycopg2
import psycopg2.errorcodes
with open('./root.crt', 'w') as f:
    f.write(bindings_list[0].get('root.crt'))
db_connection = psycopg2.connect(database=bindings_list[0].get('database'), \
    user=bindings_list[0].get('username'), \
    password=bindings_list[0].get('password'), \
    sslmode=bindings_list[0].get('sslmode'), \
    sslrootcert='./root.crt', \
    host=bindings_list[0].get('host'), \
    options=bindings_list[0].get('options'), \
    port=bindings_list[0].get('port'))
print(db_connection)

In [ ]:
import pandas as pd

cursor = db_connection.cursor()
cursor.execute("SELECT id, balance FROM accounts")
df = pd.DataFrame(cursor.fetchall(), columns = ['ID', 'Amount'])
df.head()